In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import matplotlib as mpl

mpl.style.use("seaborn")

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch data 

In [13]:
molecules = np.load("../data/molecules400.npy")
S = np.load("../data/S400.npy")
P = np.load("../data/P400.npy")

dim = 26

In [45]:
print(len(molecules[1]), S.shape, P.shape)

100 (603, 676) (603, 676)


In [19]:
from SCFInitialGuess.utilities.dataset import make_butadien_dataset

folder = "../data/"

dataset, molecules = make_butadien_dataset(
    np.load(folder + "molecules400.npy"),
    np.load(folder + "S400.npy"),
    np.load(folder + "P400.npy"),
    test_samples=100
)


[-] 2018-05-31 13:36:47: Data set normalized. Mean value std: 0.050517110993838175


# Train a network

In [40]:
from SCFInitialGuess.nn.training import Trainer
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.cost_functions import RegularizedMSE

trainer = Trainer(
    EluTrNNN([dim**2, dim**2, dim**2, dim**2]),
    cost_function=RegularizedMSE(alpha=1e-7),
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3)
)

trainer.setup()

network, sess = trainer.train(
    dataset,
    convergence_threshold=5e-7
)
    

[-] 2018-05-31 15:29:58: No target graph specified for Trainer setup. Creating new graph ...
[-] 2018-05-31 15:29:58: Setting up the training in the target graph ...
[-] 2018-05-31 15:29:58: network ...
[-] 2018-05-31 15:29:58: error function ...
[-] 2018-05-31 15:29:59: cost function ...
[-] 2018-05-31 15:29:59: training step
[-] 2018-05-31 15:29:59: Starting network training ...
[ ] 2018-05-31 15:29:59: Val. Cost: 5.766E-02. Error: 5.766E-02. Diff: 1.0E+10
[ ] 2018-05-31 15:30:01: Val. Cost: 4.784E-05. Error: 4.207E-05. Diff: 5.8E-02
[ ] 2018-05-31 15:30:04: Val. Cost: 2.498E-05. Error: 1.919E-05. Diff: 2.3E-05
[ ] 2018-05-31 15:30:07: Val. Cost: 1.512E-05. Error: 9.318E-06. Diff: 9.9E-06
[ ] 2018-05-31 15:30:09: Val. Cost: 1.304E-05. Error: 7.254E-06. Diff: 2.1E-06
[ ] 2018-05-31 15:30:12: Val. Cost: 1.972E-05. Error: 1.395E-05. Diff: 6.7E-06
[ ] 2018-05-31 15:30:15: Val. Cost: 1.090E-05. Error: 5.174E-06. Diff: 8.8E-06
[ ] 2018-05-31 15:30:17: Val. Cost: 1.023E-05. Error: 4.540E-06

In [41]:
with trainer.graph.as_default():
    p_400 = network.run(sess, dataset.testing[0])

In [42]:
from SCFInitialGuess.utilities.analysis import measure_absolute_error, statistics

statistics(measure_absolute_error(p_400, dataset))

(0.0016909562742881364, 0.0005494971231070022)

In [25]:
from os.path import join

save_path = "../data/network400.npy"
network.export(sess, save_path)

# Retrain a network

In [35]:
from SCFInitialGuess.nn.networks import EluFixedValue

structure, weights, biases = np.load(
    "../data/network.npy", 
    encoding="latin1"
)

network_300 = EluFixedValue(structure, weights, biases)
network_300.setup()

graph = tf.Graph()

with graph.as_default():
    sess_300 = tf.Session()
    network_300 = EluFixedValue(structure, weights, biases)
    network_300.setup()
    sess_300.run(tf.global_variables_initializer())
    p_300 = network_300.run(sess_300, dataset.testing[0])

In [37]:
from SCFInitialGuess.utilities.analysis import measure_absolute_error, statistics

statistics(measure_absolute_error(p_300, dataset))

(0.07300608887054842, 0.048920488103008856)